# Packages

In [10]:
%matplotlib inline
%load_ext tensorboard
!rm -rf ./logs/ 

import os
import numpy as np
import pandas as pd
import tensorflow as tf
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
from itertools import product
from sklearn.svm import SVC
from sklearn.metrics import auc, roc_curve
from sklearn.metrics import confusion_matrix
from sklearn.feature_selection import SelectPercentile
from sklearn.feature_selection import chi2, SelectKBest
from tensorboard.plugins.hparams import api as hp
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import VarianceThreshold
plt.style.use("ggplot")
matplotlib.rcParams.update({'font.size': 24})
PATH_DATA = "../datasets/CYP/"
print(tf.__version__)

2.0.0


## L model DNN

Generate DNN with L layers

In [2]:
def generate_model(layers_dim, lr, dropout, optimizer, L2):
    """layers_dim -- [n_input, n_hid_1, ..., n_output=1]"""
    hidden_layers = []
    for i in range(1,len(layers_dim)-1): hidden_layers.extend([tf.keras.layers.Dropout(dropout)] + [tf.keras.layers.Dense(layers_dim[i], activation="relu", kernel_regularizer=tf.keras.regularizers.l2(L2))])
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(layers_dim[0], activation='relu', input_shape=(layers_dim[0],))] +
        hidden_layers + 
        [tf.keras.layers.Dense(layers_dim[-1], activation="sigmoid")])
    loss_function = tf.keras.losses.BinaryCrossentropy()
    model.compile(optimizer=optimizer, loss=loss_function, metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]) 
    return model

Callbacks are useful to stop learning when some condition is reached (among other things I guess)

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('accuracy')>0.90):
          print("\n Reached 80% accuracy on training set so cancelling training!")
          self.model.stop_training = True
        if(logs.get('val_accuracy')>0.80):
          print("\n Reached 70% accuracy on validation set so cancelling training!")
          self.model.stop_training = True

callbacks = myCallback()

Example of using DNN of L layers with callbacks

In [ ]:
layers_dim = [train_data.shape[1], 15, 5, 1]
lr = 0.001
dropout = 0.2
optimizer = 'RMSprop'
L2 = 0.001

model_small_dataset =generate_model(layers_dim, lr, dropout, optimizer, L2)
#model_small_dataset.summary()

history = model_small_dataset.fit(
      train_data,train_labels,
      epochs=50,
      verbose=2,
      #validation_data = (mini_testing_2c9_data, mini_labels_testing_2c9_data)
      validation_data = (test_data, test_labels)
      )

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

## Loading data

In [ ]:
PATH_DATA = "../datasets/CYP/"

shared_data = pd.read_csv(os.path.join(PATH_DATA, "shared_set_cyp.csv"))
labels_2c9 = (shared_data["p450-cyp2c9 Activity Outcome"] == "Active").values.astype(int)
labels_3a4 = (shared_data["p450-cyp3a4 Activity Outcome"] == "Active").values.astype(int)
testing_2c9_data = pd.read_csv(os.path.join(PATH_DATA, "only_2c9_set_cyp.csv"))
labels_testing_2c9 = (testing_2c9_data["p450-cyp2c9 Activity Outcome"] == "Active").values.astype(int)
testing_3a4_data = pd.read_csv(os.path.join(PATH_DATA, "only_3a4_set_cyp.csv"))
labels_testing_3a4 = (testing_3a4_data["p450-cyp3a4 Activity Outcome"] == "Active").values.astype(int)

features_shared = np.load(os.path.join("features", "shared_set_features.npy"))
features_only_2c9 = np.load(os.path.join("features", "only_2c9_set_features.npy"))
features_only_3a4 = np.load(os.path.join("features", "only_3a4_set_features.npy"))

## Getting features (fingerprints)

There are many [fingerprints](http://rdkit.org/UGM/2012/Landrum_RDKit_UGM.Fingerprints.Final.pptx.pdf):

- Morgan
- RDKit fingerprint (to do)
- MACCS (to do)

Posar link paper que compara fingerprints (Dani)

In [ ]:
def get_features(input_sdf):
    structures_shared = Chem.SDMolSupplier(input_sdf)
    features = []
    for mol in structures_shared:
        fp = AllChem.GetMorganFingerprintAsBitVect(mol,2,nBits=1024)
        arr = np.zeros((0,), dtype=np.int8)
        DataStructs.ConvertToNumpyArray(fp,arr)
        features.append(arr)
    return np.array(features)

if os.path.exists(os.path.join("features", "shared_set_features.npy")):
    features_shared = np.load(os.path.join("features", "shared_set_features.npy"))
else:
    features_shared = get_features(os.path.join(PATH_DATA, "shared_set_cyp.sdf"))
    np.save(os.path.join("features", "shared_set_features.npy"), features_shared)

## Getting physicochemical descriptors

Molecular descriptors are widely employed to present molecular characteristics in cheminformatics. Various molecular-descriptor-calculation software programs have been developed.

- [Mordred](https://jcheminf.biomedcentral.com/articles/10.1186/s13321-018-0258-y) (To do)
- PaDEL - Descriptor
- ...


In [ ]:
from mordred import Calculator, descriptors

# create descriptor calculator with all descriptors
calc = Calculator(descriptors, ignore_3D=True)
#len(calc.descriptors)

df = pd.read_csv('CYP2C9_dataset_training.csv')
class_col = df['Class']
class_arr = class_col.to_numpy()
class_arr = np.reshape(class_arr, [class_arr.shape[0],1])
class_arr = np.squeeze(class_arr)
smi_col = df['SMILES']
smi_arr = smi_col.to_numpy()
smi_arr = np.reshape(smi_arr, [smi_arr.shape[0],1])
smi_arr = np.squeeze(smi_arr)

# calculate descriptors for a single molecule using smile
mol = Chem.MolFromSmiles(smi_arr[300])
calc(mol)[:3]

# get descriptors in a df
df_descriptors = calc.pandas(mols_short)
#df_descriptors['SLogP']




## Ferature selection

There are mainly [two ways](https://machinelearningmastery.com/feature-selection-with-categorical-data/) of chossing features of a dataset taking into account it's labels (`select_features()`):

- [chi square](https://scikit-learn.org/stable/modules/feature_selection.html#univariate-feature-selection) (`chi2`)
- [Mutual info classification](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.mutual_info_classif.html) (`mutual_info_classif`)

Also we can remove all features whose variance doesn’t meet some threshold, independently of how are they classified.
By default, it removes all zero-variance features, i.e. features that have the same value in all samples (`select_features_threshold`):

- [Removing features with low variance](https://scikit-learn.org/stable/modules/feature_selection.html#univariate-feature-selection): The smaller threshold ($p$), the more restrictive is the function, since the smaller will be the variance threshold:

    $Var_{threshold} =  p(1-p)<Var_{x} \rightarrow Accepted$

We can combine both ways to select features `select_features_comb()`:
- First we will select the features taking into account the labels of the data, using `select_features()`.
- Then we will set a variance threshold to the selected features with `select_features_threshold()`.

In [15]:
def select_features_univar(X_train, Y_train, X_test, score_func=chi2, k_best=None, percentile = None):
    """score_func=chi2 (default), mutual_info_classif"""
    if not k_best == None:
        fs = SelectKBest(score_func=score_func, k=k_best)
    elif not percentile == None:
        fs = SelectPercentile(score_func=score_func, percentile=percentile)
    else:
        print("Introduce the number of best features to be kept (`k_best`) or the percentil.")
        return
    fs.fit(X_train, Y_train)
    X_train_fs = fs.transform(X_train)
    X_test_fs = fs.transform(X_test)
    return X_train_fs, X_test_fs, fs
    
def select_features_threshold(X_train, X_test, threshold=0.9):
    """It removes all features whose variance doesn’t meet some threshold. It doesn't take into account the """
    sel = VarianceThreshold(threshold=(threshold * (1 - threshold)))
    X_train_threshold = sel.fit_transform(X_train)
    X_test_threshold = sel.transform(X_test)
    return X_train_threshold, X_test_threshold

def select_features_comb(X_train, Y_train, X_test, score_func=chi2, k_best=None, percentile = None, threshold=0.9):
    X_train_fs, X_test_fs, _ = select_features_univar(X_train, Y_train, X_test, score_func=chi2, k_best=k_best, percentile = percentile)
    X_train_comb, X_test_comb = select_features_threshold(X_train_fs, X_test_fs, threshold=threshold)
    return X_train_comb, X_test_comb

def plot_score(fs, print_scores=False):
    """plot the score for all the features"""
    if print_scores:
        for i in range(len(fs.scores_)):
            print('Feature %d: %f' % (i, fs.scores_[i]))
    plt.bar([i for i in range(len(fs.scores_))], fs.scores_)
    plt.show()
    